<a href="https://colab.research.google.com/github/Dhruva-G-Shankar/63_Hackaholics_FinTech/blob/main/Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output
import sqlite3
import hashlib
from datetime import datetime, timedelta

# BudgetTracker Class
class BudgetTracker:
    def __init__(self):
        self.income = 0
        self.expenses = pd.DataFrame(columns=["Category", "Amount", "Date"])
        self.recurring_expenses = pd.DataFrame(columns=["Category", "Amount", "Frequency"])
        self.savings_goal = 0
        self.username = None
        self.setup_database()

    def setup_database(self):
        """Create the database and tables if they don't exist."""
        self.conn = sqlite3.connect("budget_tracker.db")
        cursor = self.conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                username TEXT PRIMARY KEY,
                password TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS expenses (
                username TEXT,
                category TEXT,
                amount REAL,
                date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS recurring_expenses (
                username TEXT,
                category TEXT,
                amount REAL,
                frequency TEXT
            )
        ''')
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS savings (
                username TEXT PRIMARY KEY,
                goal REAL
            )
        ''')
        self.conn.commit()

    def hash_password(self, password):
        return hashlib.sha256(password.encode()).hexdigest()

    def register_user(self, username, password):
        hashed_pw = self.hash_password(password)
        try:
            with self.conn:
                self.conn.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_pw))
            return "User registered successfully!"
        except sqlite3.IntegrityError:
            return "Username already exists."

    def login_user(self, username, password):
        hashed_pw = self.hash_password(password)
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM users WHERE username = ? AND password = ?", (username, hashed_pw))
        user = cursor.fetchone()
        if user:
            self.username = username
            self.load_savings_goal_from_db()
            self.load_recurring_expenses()
            return "Login successful!"
        else:
            return "Invalid username or password."

    def logout_user(self):
        self.username = None
        return "Logged out successfully!"

    def add_income(self, amount):
        self.income += amount

    def add_expense(self, category, amount):
        new_expense = {"Category": category, "Amount": amount, "Date": datetime.now()}
        self.expenses = pd.concat([self.expenses, pd.DataFrame([new_expense])], ignore_index=True)

    def add_recurring_expense(self, category, amount, frequency):
        new_recurring_expense = {"Category": category, "Amount": amount, "Frequency": frequency}
        self.recurring_expenses = pd.concat([self.recurring_expenses, pd.DataFrame([new_recurring_expense])], ignore_index=True)

    def load_recurring_expenses(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT category, amount, frequency FROM recurring_expenses WHERE username = ?", (self.username,))
        rows = cursor.fetchall()
        self.recurring_expenses = pd.DataFrame(rows, columns=["Category", "Amount", "Frequency"])

    def set_savings_goal(self, amount):
        self.savings_goal = amount

    def load_savings_goal_from_db(self):
        cursor = self.conn.cursor()
        cursor.execute("SELECT goal FROM savings WHERE username = ?", (self.username,))
        row = cursor.fetchone()
        if row:
            self.savings_goal = row[0]
        else:
            self.savings_goal = 0  # Default if not set

    def save_savings_goal_to_db(self):
        """Save the savings goal to the database."""
        with self.conn:
            self.conn.execute('''
                INSERT OR REPLACE INTO savings (username, goal) VALUES (?, ?)
            ''', (self.username, self.savings_goal))

    def get_balance(self):
        total_expenses = self.expenses["Amount"].sum()
        return self.income - total_expenses

    def display_summary(self):
        balance = self.get_balance()
        progress = (balance / self.savings_goal * 100) if self.savings_goal else 0
        return {
            "Total Income": f"${self.income:.2f}",
            "Total Expenses": f"${self.expenses['Amount'].sum():.2f}",
            "Balance": f"${balance:.2f}",
            "Savings Goal": f"${self.savings_goal:.2f}",
            "Progress to Goal": f"{progress:.2f}%" if self.savings_goal else "No goal set"
        }

    def plot_expense_distribution(self):
        if self.expenses.empty:
            print("No expenses to display.")
            return
        plt.figure(figsize=(6, 6))
        expense_data = self.expenses.groupby("Category").sum()
        plt.pie(
            expense_data["Amount"],
            labels=expense_data.index,
            autopct="%1.1f%%",
            startangle=140,
            colors=sns.color_palette("pastel")
        )
        plt.title("Expense Distribution by Category")
        plt.axis("equal")
        plt.show()

    def plot_savings_progress(self):
        balance = self.get_balance()
        remaining_goal = max(self.savings_goal - balance, 0)
        values = [balance, remaining_goal]
        plt.figure(figsize=(6, 6))
        plt.pie(
            values,
            labels=["Current Balance", "Remaining Goal"],
            colors=["green", "lightgray"],
            startangle=90,
            counterclock=False,
            wedgeprops={"width": 0.3}
        )
        plt.title("Savings Progress")
        plt.axis("equal")
        plt.show()

# Simple Budget Tracker UI
def create_budget_tracker_ui():
    tracker = BudgetTracker()

    # Feedback output area
    feedback_output = widgets.Output()
    visualization_output = widgets.Output()

    # Login/Registration Widgets
    username_input = widgets.Text(description="Username:")
    password_input = widgets.Password(description="Password:")
    register_button = widgets.Button(description="Register")
    login_button = widgets.Button(description="Login")
    logout_button = widgets.Button(description="Logout", disabled=True)

    # Income, Expense, and Savings Goal Widgets
    income_input = widgets.FloatText(value=0, description="Add Income:", disabled=True)
    expense_category_input = widgets.Dropdown(
        options=["Rent", "Utilities", "Groceries", "Entertainment", "Other"],
        description="Category:", disabled=True
    )
    expense_amount_input = widgets.FloatText(value=0, description="Expense:", disabled=True)
    savings_goal_input = widgets.FloatText(value=0, description="Savings Goal:", disabled=True)

    add_income_button = widgets.Button(description="Add Income", disabled=True)
    add_expense_button = widgets.Button(description="Add Expense", disabled=True)
    set_savings_goal_button = widgets.Button(description="Set Goal", disabled=True)
    summary_button = widgets.Button(description="Show Summary", disabled=True)
    charts_button = widgets.Button(description="Show Charts", disabled=True)

    # Recurring Expense Widgets
    recurring_category_input = widgets.Dropdown(
        options=["Rent", "Utilities", "Groceries", "Entertainment", "Other"],
        description="Category:"
    )
    recurring_amount_input = widgets.FloatText(value=0, description="Amount:")
    recurring_frequency_input = widgets.Dropdown(
        options=["Daily", "Weekly", "Monthly"],
        description="Frequency:"
    )
    add_recurring_button = widgets.Button(description="Add Recurring Expense", disabled=True)

    # Action Widgets
    actions_box = widgets.VBox([
        income_input, add_income_button,
        expense_category_input, expense_amount_input, add_expense_button,
        savings_goal_input, set_savings_goal_button,
        summary_button, charts_button,
        recurring_category_input, recurring_amount_input, recurring_frequency_input, add_recurring_button
    ])

    # Enable/Disable Main UI based on login
    def enable_main_ui():
        logout_button.disabled = False
        for widget in actions_box.children:
            widget.disabled = False

    def disable_main_ui():
        logout_button.disabled = True
        for widget in actions_box.children:
            widget.disabled = True

    def handle_register(b):
        feedback = tracker.register_user(username_input.value, password_input.value)
        with feedback_output:
            clear_output(wait=True)
            print(feedback)

    def handle_login(b):
        feedback = tracker.login_user(username_input.value, password_input.value)
        with feedback_output:
            clear_output(wait=True)
            print(feedback)
        if tracker.username:
            enable_main_ui()

    def handle_logout(b):
        tracker.save_savings_goal_to_db()
        feedback = tracker.logout_user()
        with feedback_output:
            clear_output(wait=True)
            print(feedback)
        disable_main_ui()

    def handle_add_income(b):
        tracker.add_income(income_input.value)
        with feedback_output:
            clear_output(wait=True)
            print(f"Income added! New Balance: ${tracker.get_balance():.2f}")
        income_input.value = 0

    def handle_add_expense(b):
        tracker.add_expense(expense_category_input.value, expense_amount_input.value)
        with feedback_output:
            clear_output(wait=True)
            print(f"Expense added! New Balance: ${tracker.get_balance():.2f}")
        expense_amount_input.value = 0

    def handle_set_savings_goal(b):
        tracker.set_savings_goal(savings_goal_input.value)
        with feedback_output:
            clear_output(wait=True)
            print(f"Savings goal set to: ${tracker.savings_goal:.2f}")
        savings_goal_input.value = 0

    def handle_show_summary(b):
        summary = tracker.display_summary()
        with feedback_output:
            clear_output(wait=True)
            for key, value in summary.items():
                print(f"{key}: {value}")

    def handle_show_charts(b):
        with visualization_output:
            clear_output(wait=True)
            tracker.plot_expense_distribution()
            tracker.plot_savings_progress()

    def handle_add_recurring_expense(b):
        tracker.add_recurring_expense(
            recurring_category_input.value,
            recurring_amount_input.value,
            recurring_frequency_input.value
        )
        with feedback_output:
            clear_output(wait=True)
            print("Recurring expense added.")

    # Attach button click handlers
    register_button.on_click(handle_register)
    login_button.on_click(handle_login)
    logout_button.on_click(handle_logout)
    add_income_button.on_click(handle_add_income)
    add_expense_button.on_click(handle_add_expense)
    set_savings_goal_button.on_click(handle_set_savings_goal)
    summary_button.on_click(handle_show_summary)
    charts_button.on_click(handle_show_charts)
    add_recurring_button.on_click(handle_add_recurring_expense)

    # Layout of UI components
    ui_layout = widgets.VBox([
        username_input, password_input, login_button, register_button, logout_button,
        actions_box,
        feedback_output,
        visualization_output
    ])
    display(ui_layout)

create_budget_tracker_ui()


<ipython-input-2-94b52fa7fcf0>:89: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.expenses = pd.concat([self.expenses, pd.DataFrame([new_expense])], ignore_index=True)
<ipython-input-2-94b52fa7fcf0>:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.recurring_expenses = pd.concat([self.recurring_expenses, pd.DataFrame([new_recurring_expense])], ignore_index=True)


<Figure size 600x600 with 0 Axes>

<Figure size 600x600 with 0 Axes>

<Figure size 600x600 with 0 Axes>